In [256]:
import numpy as np
import pandas as pd
import scipy
from scipy.optimize import minimize
import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn import linear_model as lm
from sklearn.feature_selection import r_regression
from sklearn.metrics import r2_score

In [257]:
import warnings
warnings.filterwarnings('ignore')

In [258]:
approval = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vS-FKWVTTFtJT6u56e0bqdfoMcXvDO1DUChsJ3jQAMB2lZk2SMqVfmg7dGjclTYkYWz-Pm5lfcLPjp4/pub?output=csv')
# approval = approval.reset_index()
# approval = approval.rename({'index':'poll_id'}, axis=1)
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,tracking,approve,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp
0,Donald Trump (2nd term),All polls,Ipsos,Reuters,6/11/2025,6/16/2025,4258.0,A,1.354591,1.277143,NaN,42.0,54.0,-12.0,43.90702,53.049230,-9.142208,https://www.reuters.com/world/us/trump-approva...,6/16/2025,20:00:28 17 Jun 2025
1,Donald Trump (2nd term),Immigration,Ipsos,Reuters,6/11/2025,6/16/2025,4258.0,A,1.354591,1.277143,NaN,44.0,49.0,-5.0,45.26391,49.639470,-4.375561,https://www.reuters.com/world/us/trump-approva...,6/16/2025,20:00:43 17 Jun 2025
2,Donald Trump (2nd term),Economy,Ipsos,Reuters,6/11/2025,6/16/2025,4258.0,A,1.354591,1.277143,NaN,39.0,52.0,-13.0,41.46937,52.443130,-10.973760,https://www.reuters.com/world/us/trump-approva...,6/16/2025,20:00:41 17 Jun 2025
3,Donald Trump (2nd term),Adults,Ipsos,Reuters,6/11/2025,6/16/2025,4258.0,A,1.354591,1.277143,NaN,42.0,54.0,-12.0,43.02093,53.878180,-10.857250,https://www.reuters.com/world/us/trump-approva...,6/16/2025,20:00:35 17 Jun 2025
4,Donald Trump (2nd term),Trade,Harris Insights & Analytics,Harvard CAPS,6/11/2025,6/12/2025,2097.0,RV,1.318091,1.167179,NaN,41.0,52.0,-11.0,40.41073,53.541910,-13.131180,https://harvardharrispoll.com/wp-content/uploa...,6/12/2025,20:00:46 17 Jun 2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558,Donald Trump (2nd term),Adults,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,NaN,47.0,41.0,6.0,48.02093,40.878180,7.142750,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,20:00:35 17 Jun 2025
1559,Donald Trump (2nd term),Cost,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.996709,0.000000,NaN,35.0,41.0,-6.0,38.40033,40.752300,-2.351971,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,20:00:45 17 Jun 2025
1560,Donald Trump (2nd term),Weak,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,NaN,15.0,8.0,7.0,13.05687,7.212527,5.844344,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,20:00:39 17 Jun 2025
1561,Donald Trump (2nd term),All polls,InsiderAdvantage,NaN,1/20/2025,1/20/2025,800.0,RV,1.093180,0.000000,NaN,56.0,39.0,17.0,53.69146,40.814340,12.877120,https://insideradvantage.com/first-insideradva...,1/20/2025,20:00:28 17 Jun 2025


In [259]:
approval = approval[~approval['subgroup'].isin(['Weak', 'Strong', 'Economy', 'Cost', 'Immigration', 'Trade', 'Adults', 'Voters'])]

In [260]:
approval.to_csv('trump_approval_polls.csv')

In [261]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'enddate', 'samplesize', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp'], dtype=object)

In [262]:
approval.shape

(339, 20)

In [263]:
# dates = approval['Dates'].str.extract(r'(\d{1,2}\/\d{1,2}) \- (\d{1,2}\/\d{1,2}), (\d{4})')
# dates

In [264]:
# dates['end_date'] = dates[1] + '/' + dates[2]
# dates['end_date'] = pd.to_datetime(dates['end_date'])
# dates

In [265]:
# approval['Dates'] = dates['end_date']
# approval.head()

In [266]:
# samples = approval['Sample'].str.extract(r'[0-9,]{3,5} (\w{1,2})@@(\d{3,4})')
# samples

In [267]:
# approval['population'] = samples[0]
# approval['sample_size'] = samples[1].astype(int)
# approval.head()

In [268]:
ratings = pd.read_csv('pollster_ratings_silver.csv')
ratings

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
0,AtlasIntel,A+@@1,-0.957740,D+0.16@@0.16334188,91,D+
1,Marquette University Law School,A/B@@4,-0.920261,D+0.32@@0.31545374,17,D+
2,The Washington Post,A+@@1,-0.902281,D+0.90@@0.89811021,98,D+
3,Research & Polling Inc.,A@@2,-0.871136,R+0.55@@-0.54944503,44,R+
4,The New York Times/Siena College,A@@2,-0.763947,D+1.41@@1.4121077,116,D+
...,...,...,...,...,...,...
530,Nielson Brothers Polling,C/D@@12,1.946864,D+3.05@@3.0515239,11,D+
531,Ascend Action,C/D@@12,2.282717,D+3.35@@3.3485439,6,D+
532,Jayhawk Consulting Services,C/D@@12,2.353968,D+2.27@@2.2674108,2,D+
533,Fort Hays State University,C/D@@12,2.489170,R+1.10@@-1.0960908,6,R+


In [269]:
ratings['Pollster'].values

array(['AtlasIntel', 'Marquette University Law School',
       'The Washington Post', 'Research & Polling Inc.',
       'The New York Times/Siena College', 'Cygnal', 'SurveyUSA',
       'Siena College', 'Beacon Research/Shaw & Co. Research',
       'Alaska Survey Research', 'OnMessage Inc.', 'TIPP Insights',
       'Emerson College', 'Landmark Communications',
       'Monmouth University', 'Data Orbital',
       'University of North Florida', 'MassINC Polling Group',
       'Marist College', 'Public Policy Institute of California',
       'Public Policy Polling', 'University of California Berkeley',
       'CBS News/The New York Times', 'Christopher Newport University',
       'Mason-Dixon Polling & Strategy', 'Strategic Vision LLC',
       'Suffolk University', 'Stockton University',
       'Field Research Corp. (Field Poll)', 'Quinnipiac University',
       'Los Angeles Times',
       'Hart Research Associates/Public Opinion Strategies',
       'Public Opinion Strategies', 'Susquehan

In [270]:
ratings[ratings['Pollster'].str.contains('North Star')]#.sum()

,Pollster,Grade,Predictive Plus-Minus,Mean-reverted bias,Number of polls,cat
75,North Star Opinion Research,B/C@@8,0.043339,R+0.63@@-0.62697512,13,R+
425,Univision/Latino Decisions/North Star Opinion ...,B/C@@8,0.830409,D+1.06@@1.0564657,7,D+


In [271]:
banned_pollsters = ratings[ratings['Grade'].str.contains('F')]['Pollster'].values
banned_pollsters

array(['Strategic Vision LLC', 'Pharos Research Group', 'Research 2000',
       'Big Data Poll', 'Overtime Politics', 'Rethink Priorities',
       'Blumenthal Research Daily', 'CSP Polling', 'KG Polling',
       'OurProgress (The Progress Campaign)', 'TCJ Research'],
      dtype=object)

In [272]:
approval = approval[~approval['pollster'].isin(banned_pollsters)]
approval.shape

(339, 20)

In [273]:
approval = pd.merge(left=approval, right=ratings[['Pollster', 'Predictive Plus-Minus']], left_on='pollster',
                   right_on='Pollster', how='inner')
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),All polls,Ipsos,Reuters,6/11/2025,6/16/2025,4258.0,A,1.354591,1.277143,...,54.0,-12.000000,43.90702,53.04923,-9.142208,https://www.reuters.com/world/us/trump-approva...,6/16/2025,20:00:28 17 Jun 2025,Ipsos,0.442898
1,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,6/11/2025,6/12/2025,2097.0,RV,1.318091,1.091724,...,50.0,-4.000000,44.72704,51.85780,-7.130760,https://harvardharrispoll.com/wp-content/uploa...,6/12/2025,20:00:28 17 Jun 2025,Harris Insights & Analytics,0.162447
2,Donald Trump (2nd term),All polls,TIPP Insights,League of American Workers,6/9/2025,6/11/2025,1584.0,RV,1.504667,1.066622,...,50.0,-7.000000,44.35295,51.94475,-7.591801,https://tippinsights.com/league-of-american-wo...,6/11/2025,20:00:28 17 Jun 2025,TIPP Insights,-0.571372
3,Donald Trump (2nd term),All polls,YouGov,Economist,6/13/2025,6/16/2025,1512.0,A,0.839664,0.839664,...,54.0,-13.000000,41.80408,54.43353,-12.629450,https://d3nkl3psvxxpe9.cloudfront.net/document...,6/16/2025,20:00:28 17 Jun 2025,YouGov,0.047635
4,Donald Trump (2nd term),All polls,J.L. Partners,NaN,6/10/2025,6/11/2025,1807.0,RV,1.021452,0.784589,...,46.0,-4.000000,42.77581,49.10620,-6.330391,https://www.dailymail.co.uk/news/article-14803...,6/11/2025,20:00:28 17 Jun 2025,J.L. Partners,0.197911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500.0,LV,0.213379,0.000000,...,43.0,9.000000,49.05586,44.59280,4.463058,https://www.rasmussenreports.com/public_conten...,1/24/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015
319,Donald Trump (2nd term),All polls,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.0,18.000000,53.83864,41.27032,12.568320,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,20:00:28 17 Jun 2025,RMG Research,0.278023
320,Donald Trump (2nd term),All polls,AtlasIntel,NaN,1/21/2025,1/23/2025,1882.0,A,1.910964,0.000000,...,49.7,0.399998,49.91161,48.34523,1.566380,https://cdn.atlasintel.org/ed3f5238-e823-495e-...,1/23/2025,20:00:28 17 Jun 2025,AtlasIntel,-0.957740
321,Donald Trump (2nd term),All polls,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,...,41.0,6.000000,48.90702,40.04923,8.857792,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,20:00:28 17 Jun 2025,Ipsos,0.442898


In [274]:
approval.shape

(323, 22)

In [275]:
approval['Pollster'].value_counts()

Pollster
Rasmussen Reports                                     103
YouGov                                                 40
Morning Consult                                        26
RMG Research                                           21
Ipsos                                                  15
J.L. Partners                                          10
TIPP Insights                                           8
Global Strategy Group/GBAO (Navigator Research)         7
Harris Insights & Analytics                             7
Echelon Insights                                        6
Quinnipiac University                                   5
AtlasIntel                                              5
Emerson College                                         5
Gallup                                                  5
McLaughlin & Associates                                 4
SSRS                                                    4
American Research Group                                 4
Clari

In [276]:
# approval[~approval['tracking'].isna()]

In [277]:
approval['tracking'].value_counts()

tracking
True    107
Name: count, dtype: int64

In [278]:
# full_approval = approval.copy()

In [279]:
approval = approval[(~approval['samplesize'].isna())]
approval

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,disapprove,net,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus
0,Donald Trump (2nd term),All polls,Ipsos,Reuters,6/11/2025,6/16/2025,4258.0,A,1.354591,1.277143,...,54.0,-12.000000,43.90702,53.04923,-9.142208,https://www.reuters.com/world/us/trump-approva...,6/16/2025,20:00:28 17 Jun 2025,Ipsos,0.442898
1,Donald Trump (2nd term),All polls,Harris Insights & Analytics,Harvard CAPS,6/11/2025,6/12/2025,2097.0,RV,1.318091,1.091724,...,50.0,-4.000000,44.72704,51.85780,-7.130760,https://harvardharrispoll.com/wp-content/uploa...,6/12/2025,20:00:28 17 Jun 2025,Harris Insights & Analytics,0.162447
2,Donald Trump (2nd term),All polls,TIPP Insights,League of American Workers,6/9/2025,6/11/2025,1584.0,RV,1.504667,1.066622,...,50.0,-7.000000,44.35295,51.94475,-7.591801,https://tippinsights.com/league-of-american-wo...,6/11/2025,20:00:28 17 Jun 2025,TIPP Insights,-0.571372
3,Donald Trump (2nd term),All polls,YouGov,Economist,6/13/2025,6/16/2025,1512.0,A,0.839664,0.839664,...,54.0,-13.000000,41.80408,54.43353,-12.629450,https://d3nkl3psvxxpe9.cloudfront.net/document...,6/16/2025,20:00:28 17 Jun 2025,YouGov,0.047635
4,Donald Trump (2nd term),All polls,J.L. Partners,NaN,6/10/2025,6/11/2025,1807.0,RV,1.021452,0.784589,...,46.0,-4.000000,42.77581,49.10620,-6.330391,https://www.dailymail.co.uk/news/article-14803...,6/11/2025,20:00:28 17 Jun 2025,J.L. Partners,0.197911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,1/20/2025,1/24/2025,1500.0,LV,0.213379,0.000000,...,43.0,9.000000,49.05586,44.59280,4.463058,https://www.rasmussenreports.com/public_conten...,1/24/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015
319,Donald Trump (2nd term),All polls,RMG Research,Napolitan News Service,1/20/2025,1/23/2025,3000.0,RV,0.927446,0.000000,...,39.0,18.000000,53.83864,41.27032,12.568320,https://x.com/ScottWRasmussen/status/188280455...,1/23/2025,20:00:28 17 Jun 2025,RMG Research,0.278023
320,Donald Trump (2nd term),All polls,AtlasIntel,NaN,1/21/2025,1/23/2025,1882.0,A,1.910964,0.000000,...,49.7,0.399998,49.91161,48.34523,1.566380,https://cdn.atlasintel.org/ed3f5238-e823-495e-...,1/23/2025,20:00:28 17 Jun 2025,AtlasIntel,-0.957740
321,Donald Trump (2nd term),All polls,Ipsos,Reuters,1/20/2025,1/21/2025,1077.0,A,0.749248,0.000000,...,41.0,6.000000,48.90702,40.04923,8.857792,https://www.ipsos.com/en-us/reuters-ipsos-trum...,1/21/2025,20:00:28 17 Jun 2025,Ipsos,0.442898


In [280]:
auxiliary = pd.read_csv('auxiliary_pollster_rating_stats.csv')
auxiliary

,Pollster,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat
0,AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+
1,The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+
2,Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+
3,The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+
4,Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+
...,...,...,...,...,...,...,...,...
93,Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+
94,Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+
95,Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+
96,McLaughlin & Associates,0.689655,34.5%@@65.517241,R+4.94@@-4.9392128,6.2@@6.17804,0.000000,1,R+


In [281]:
house_eff = auxiliary['House effect'].str.extract(r'(\w)\+(\d\.\d\d)@@*.')
house_eff[1] = house_eff[1].astype(float)
house_eff

,0,1
0,R,2.00
1,D,1.02
2,D,0.63
3,D,0.79
4,R,0.60
...,...,...
93,R,2.44
94,R,0.45
95,R,3.23
96,R,4.94


In [282]:
# R --> -1, D --> 1
house_eff[0] = house_eff[0].replace({'R':-1, 'D':1})
house_eff['house_effect'] = house_eff[0] * house_eff[1]
house_eff

,0,1,house_effect
0,-1,2.00,-2.00
1,1,1.02,1.02
2,1,0.63,0.63
3,1,0.79,0.79
4,-1,0.60,-0.60
...,...,...,...
93,-1,2.44,-2.44
94,-1,0.45,-0.45
95,-1,3.23,-3.23
96,-1,4.94,-4.94


In [283]:
auxiliary['house_effect'] = house_eff['house_effect']
auxiliary = auxiliary.set_index(['Pollster'])
auxiliary

,Correct calls,Outside MoE,House effect,ADPA,Herding penalty,herd_cat,p_cat,house_effect
Pollster,,,,,,,,
AtlasIntel,0.840659,8.8%@@91.208791,R+2.00@@-1.9990723,3.7@@3.6687982,0.096047,2,R+,-2.00
The Washington Post,0.693878,6.1%@@93.877551,D+1.02@@1.0173736,3.6@@3.5756676,0.221841,3,D+,1.02
Research & Polling Inc.,0.886364,9.1%@@90.9090906,D+0.63@@0.63289553,4.6@@4.642076,0.286276,3,D+,0.63
The New York Times/Siena College,0.737069,18.1%@@81.896552,D+0.79@@0.79133403,3.2@@3.2483816,0.280332,3,D+,0.79
Cygnal,0.939655,13.8%@@86.206897,R+0.60@@-0.59704119,3.5@@3.4644768,0.521310,4,R+,-0.60
...,...,...,...,...,...,...,...,...
Digital Research,0.904762,38.1%@@61.904761,R+2.44@@-2.4431372,11.4@@11.413808,0.000000,1,R+,-2.44
Mitchell Research & Communications,0.695122,39%@@60.975611,R+0.45@@-0.44617203,4.4@@4.4410443,0.229193,3,R+,-0.45
Merriman River Group,0.840000,60%@@39.999998,R+3.23@@-3.2347174,10.3@@10.260222,0.000000,1,R+,-3.23


In [284]:
def apply_house_eff(data, pollster):
    df = data[data['pollster']==pollster]
    try:
        df['approve_adj'] = df['approve'] + (auxiliary.loc[pollster, 'house_effect'] / 2)
        df['disapprove_adj'] = df['disapprove'] - (auxiliary.loc[pollster, 'house_effect'] / 2)
    except KeyError:
        df['approve_adj'] = df['approve']
        df['disapprove_adj'] = df['disapprove']
    return df

In [285]:
pollsters = approval['pollster'].value_counts().index.values
df_adjs = []
for p in pollsters:
    df = apply_house_eff(approval, p)
    df_adjs.append(df)

approval_adj = pd.concat(df_adjs, axis=0)
approval_adj

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_approve,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj
6,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/10/2025,6/16/2025,1500.0,LV,0.842577,0.740198,...,49.05586,48.59280,0.463058,https://www.rasmussenreports.com/public_conten...,6/16/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,51.360,47.640
19,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/6/2025,6/12/2025,1500.0,LV,0.331738,0.210363,...,50.05586,46.59280,3.463058,https://www.rasmussenreports.com/public_conten...,6/12/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,52.360,45.640
23,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/9/2025,6/13/2025,1500.0,LV,0.161067,0.121868,...,49.05586,47.59280,1.463058,https://www.rasmussenreports.com/public_conten...,6/13/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,51.360,46.640
25,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,5/30/2025,6/5/2025,1500.0,LV,0.322285,0.084555,...,47.05586,50.59280,-3.536942,https://www.rasmussenreports.com/public_conten...,6/5/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,49.360,49.640
26,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/5/2025,6/11/2025,1500.0,LV,0.109627,0.062635,...,50.05586,46.59280,3.463058,https://www.rasmussenreports.com/public_conten...,6/11/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,52.360,45.640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Donald Trump (2nd term),All polls,Remington Research Group,Humanity for Freedom Foundation,3/15/2025,3/18/2025,800.0,LV,0.999693,0.000000,...,49.16477,48.61156,0.553207,https://www.humanityforfreedomfoundation.org/f...,3/18/2025,20:00:28 17 Jun 2025,Remington Research Group,0.322523,49.295,48.705
214,Donald Trump (2nd term),All polls,North Star Opinion Research,Defending Democracy Together,3/13/2025,3/17/2025,1000.0,RV,1.158492,0.000000,...,46.28000,50.75776,-4.477760,https://www.defendingdemocracytogether.org/app...,3/17/2025,20:00:28 17 Jun 2025,North Star Opinion Research,0.043339,46.000,51.000
273,Donald Trump (2nd term),All polls,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2/19/2025,1000.0,LV,1.404286,0.000000,...,49.50373,48.58903,0.914700,https://onmessageinc.com/wp-content/uploads/20...,2/19/2025,20:00:28 17 Jun 2025,OnMessage Inc.,-0.600249,48.225,50.775
287,Donald Trump (2nd term),All polls,SurveyUSA,NaN,2/13/2025,2/16/2025,2000.0,A,1.824873,0.000000,...,50.24015,45.72530,4.514851,https://www.surveyusa.com/client/PollReport.as...,2/16/2025,20:00:28 17 Jun 2025,SurveyUSA,-0.732433,50.765,45.235


In [286]:
approval.shape

(323, 22)

In [287]:
approval = approval_adj.copy()
approval.shape

(323, 24)

In [288]:
approval['poll_id'] = pd.factorize(approval['pollster'] + approval['enddate'])[0]
approval.head(7)

,president,subgroup,pollster,sponsors,startdate,enddate,samplesize,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,Predictive Plus-Minus,approve_adj,disapprove_adj,poll_id
6,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/10/2025,6/16/2025,1500.0,LV,0.842577,0.740198,...,48.5928,0.463058,https://www.rasmussenreports.com/public_conten...,6/16/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,51.36,47.64,0
19,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/6/2025,6/12/2025,1500.0,LV,0.331738,0.210363,...,46.5928,3.463058,https://www.rasmussenreports.com/public_conten...,6/12/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,52.36,45.64,1
23,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/9/2025,6/13/2025,1500.0,LV,0.161067,0.121868,...,47.5928,1.463058,https://www.rasmussenreports.com/public_conten...,6/13/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,51.36,46.64,2
25,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,5/30/2025,6/5/2025,1500.0,LV,0.322285,0.084555,...,50.5928,-3.536942,https://www.rasmussenreports.com/public_conten...,6/5/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,49.36,49.64,3
26,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/5/2025,6/11/2025,1500.0,LV,0.109627,0.062635,...,46.5928,3.463058,https://www.rasmussenreports.com/public_conten...,6/11/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,52.36,45.64,4
28,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/4/2025,6/10/2025,1500.0,LV,0.108911,0.055330,...,48.5928,0.463058,https://www.rasmussenreports.com/public_conten...,6/10/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,51.36,47.64,5
30,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/3/2025,6/9/2025,1500.0,LV,0.108630,0.048174,...,49.5928,-1.536942,https://www.rasmussenreports.com/public_conten...,6/9/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,50.36,48.64,6


In [289]:
approval['population'] = approval['population'].str.strip(' ')

In [290]:
def pipeline(data: pd.DataFrame):
    dup = data[data.duplicated(subset='poll_id', keep=False)]
    ndup = data[~data.duplicated(subset='poll_id', keep=False)]
    # print('dup', dup, '\n ndup', ndup)
    dup_a = dup[dup['population'].isin(['A'])]
    # print('dup_a', dup_a)
    df = pd.concat([dup_a, ndup], axis=0)
    
    dup = df[df.duplicated(subset='poll_id', keep=False)]
    ndup = df[~df.duplicated(subset='poll_id', keep=False)]
    dup_lv = dup[dup['population'].isin(['RV'])]
    # print('dup_rv', dup_lv, 'ndup', ndup)
    df = pd.concat([dup_lv, ndup], axis=0)
    
    df['end_date'] = pd.to_datetime(df['end_date'])
    
    return df

In [291]:
approval = approval.rename({'enddate':'end_date', 'samplesize':'sample_size', 'Predictive Plus-Minus':'pred_plus_minus'}, axis=1)

In [292]:
approval['population'].value_counts().index.values

array(['LV', 'RV', 'A', 'V'], dtype=object)

In [293]:
approval.shape, pipeline(approval).shape

((323, 25), (313, 25))

In [294]:
pipeline(approval)

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
6,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/10/2025,2025-06-16,1500.0,LV,0.842577,0.740198,...,48.59280,0.463058,https://www.rasmussenreports.com/public_conten...,6/16/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,51.360,47.640,0
19,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/6/2025,2025-06-12,1500.0,LV,0.331738,0.210363,...,46.59280,3.463058,https://www.rasmussenreports.com/public_conten...,6/12/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,52.360,45.640,1
23,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/9/2025,2025-06-13,1500.0,LV,0.161067,0.121868,...,47.59280,1.463058,https://www.rasmussenreports.com/public_conten...,6/13/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,51.360,46.640,2
25,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,5/30/2025,2025-06-05,1500.0,LV,0.322285,0.084555,...,50.59280,-3.536942,https://www.rasmussenreports.com/public_conten...,6/5/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,49.360,49.640,3
26,Donald Trump (2nd term),All polls,Rasmussen Reports,NaN,6/5/2025,2025-06-11,1500.0,LV,0.109627,0.062635,...,46.59280,3.463058,https://www.rasmussenreports.com/public_conten...,6/11/2025,20:00:28 17 Jun 2025,Rasmussen Reports,0.087015,52.360,45.640,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Donald Trump (2nd term),All polls,Remington Research Group,Humanity for Freedom Foundation,3/15/2025,2025-03-18,800.0,LV,0.999693,0.000000,...,48.61156,0.553207,https://www.humanityforfreedomfoundation.org/f...,3/18/2025,20:00:28 17 Jun 2025,Remington Research Group,0.322523,49.295,48.705,313
214,Donald Trump (2nd term),All polls,North Star Opinion Research,Defending Democracy Together,3/13/2025,2025-03-17,1000.0,RV,1.158492,0.000000,...,50.75776,-4.477760,https://www.defendingdemocracytogether.org/app...,3/17/2025,20:00:28 17 Jun 2025,North Star Opinion Research,0.043339,46.000,51.000,314
273,Donald Trump (2nd term),All polls,OnMessage Inc.,Rescuing the American Dream,2/17/2025,2025-02-19,1000.0,LV,1.404286,0.000000,...,48.58903,0.914700,https://onmessageinc.com/wp-content/uploads/20...,2/19/2025,20:00:28 17 Jun 2025,OnMessage Inc.,-0.600249,48.225,50.775,315
287,Donald Trump (2nd term),All polls,SurveyUSA,NaN,2/13/2025,2025-02-16,2000.0,A,1.824873,0.000000,...,45.72530,4.514851,https://www.surveyusa.com/client/PollReport.as...,2/16/2025,20:00:28 17 Jun 2025,SurveyUSA,-0.732433,50.765,45.235,316


In [295]:
# approval[approval['pollster']=='Harris Insights & Analytics']

In [296]:
full_approval = approval.copy()

In [297]:
approval = approval[approval['tracking'].isna()]

In [298]:
# banned_pollsters_aux = ['Rasmussen Reports', 'co/efficient', 'TIPP Insights', 'Civiqs', 'Trafalgar Group', 'OnMessage Inc.']
banned_pollsters_aux = pd.read_csv('banned_pollsters_aux.csv')['banned_pollsters'].values
banned_pollsters_aux

array(['Rasmussen Reports', 'TIPP Insights', 'Trafalgar Group',
       'OnMessage Inc.', 'McLaughlin & Associates',
       'North Star Opinion Research', 'Trafalgar Group/Insider Advantage'],
      dtype=object)

In [299]:
approval = approval[~approval['pollster'].isin(banned_pollsters_aux)]
approval.head()

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
3,Donald Trump (2nd term),All polls,YouGov,Economist,6/13/2025,6/16/2025,1512.0,A,0.839664,0.839664,...,54.43353,-12.629450,https://d3nkl3psvxxpe9.cloudfront.net/document...,6/16/2025,20:00:28 17 Jun 2025,YouGov,0.047635,41.25,53.75,102
13,Donald Trump (2nd term),All polls,YouGov,Economist,6/6/2025,6/9/2025,1533.0,A,0.636911,0.360310,...,52.43353,-8.629448,https://d3nkl3psvxxpe9.cloudfront.net/document...,6/9/2025,20:00:28 17 Jun 2025,YouGov,0.047635,43.25,51.75,103
16,Donald Trump (2nd term),All polls,YouGov,CBS News,6/4/2025,6/6/2025,2428.0,A,0.727909,0.270644,...,55.43353,-9.629448,https://www.cbsnews.com/news/deportation-immig...,6/6/2025,20:00:28 17 Jun 2025,YouGov,0.047635,45.25,54.75,104
22,Donald Trump (2nd term),All polls,YouGov,Economist,5/30/2025,6/2/2025,1610.0,A,0.583608,0.139243,...,49.43353,-3.629448,https://d3nkl3psvxxpe9.cloudfront.net/document...,6/2/2025,20:00:28 17 Jun 2025,YouGov,0.047635,45.25,48.75,105
32,Donald Trump (2nd term),All polls,YouGov,Economist,5/23/2025,5/26/2025,1660.0,A,0.565511,0.041857,...,52.43353,-7.629448,https://d3nkl3psvxxpe9.cloudfront.net/document...,5/26/2025,20:00:28 17 Jun 2025,YouGov,0.047635,44.25,51.75,106


In [345]:
approval[approval['pollster']=='Civiqs']# [['approve', 'disapprove', 'approve_adj', 'disapprove_adj']]

,president,subgroup,pollster,sponsors,startdate,end_date,sample_size,population,weight,influence,...,adjusted_disapprove,adjusted_net,url,createddate,timestamp,Pollster,pred_plus_minus,approve_adj,disapprove_adj,poll_id
29,Donald Trump (2nd term),All polls,Civiqs,Daily Kos,5/17/2025,5/20/2025,1018.0,RV,1.120966,0.048214,...,51.26089,-4.63744,https://civiqs.com/documents/Civiqs_DailyKos_b...,5/20/2025,20:00:28 17 Jun 2025,Civiqs,0.158562,47.86,51.14,282
143,Donald Trump (2nd term),All polls,Civiqs,Daily Kos,4/12/2025,4/15/2025,1124.0,RV,1.149724,0.000000,...,52.26089,-6.63744,https://civiqs.com/reports/2025/04/17/report-a...,4/15/2025,20:00:28 17 Jun 2025,Civiqs,0.158562,46.86,52.14,283
181,Donald Trump (2nd term),All polls,Civiqs,Daily Kos,2/28/2025,3/3/2025,1031.0,RV,1.124632,0.000000,...,51.26089,-3.63744,https://civiqs.com/reports/2025/03/04/report-a...,3/3/2025,20:00:28 17 Jun 2025,Civiqs,0.158562,48.86,51.14,284


In [301]:
approval.columns.values

array(['president', 'subgroup', 'pollster', 'sponsors', 'startdate',
       'end_date', 'sample_size', 'population', 'weight', 'influence',
       'tracking', 'approve', 'disapprove', 'net', 'adjusted_approve',
       'adjusted_disapprove', 'adjusted_net', 'url', 'createddate',
       'timestamp', 'Pollster', 'pred_plus_minus', 'approve_adj',
       'disapprove_adj', 'poll_id'], dtype=object)

In [302]:
approval_polls = approval[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'approve_adj', 'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id']]
approval_polls

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id
3,YouGov,Economist,6/13/2025,6/16/2025,1512.0,A,41.0,54.0,41.250,53.750,-13.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,102
13,YouGov,Economist,6/6/2025,6/9/2025,1533.0,A,43.0,52.0,43.250,51.750,-9.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,103
16,YouGov,CBS News,6/4/2025,6/6/2025,2428.0,A,45.0,55.0,45.250,54.750,-10.0,https://www.cbsnews.com/news/deportation-immig...,0.047635,104
22,YouGov,Economist,5/30/2025,6/2/2025,1610.0,A,45.0,49.0,45.250,48.750,-4.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,105
32,YouGov,Economist,5/23/2025,5/26/2025,1660.0,A,44.0,52.0,44.250,51.750,-8.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,Public Religion Research Institute,NaN,2/28/2025,3/20/2025,5025.0,A,43.0,54.0,43.000,54.000,-11.0,https://www.prri.org/wp-content/uploads/2025/0...,0.103864,311
204,Public Opinion Strategies,Paragon Health Institute,2/25/2025,3/2/2025,1000.0,RV,51.0,49.0,48.930,51.070,2.0,https://paragoninstitute.org/medicaid/paragon-...,-0.246974,312
210,Remington Research Group,Humanity for Freedom Foundation,3/15/2025,3/18/2025,800.0,LV,50.0,48.0,49.295,48.705,2.0,https://www.humanityforfreedomfoundation.org/f...,0.322523,313
287,SurveyUSA,NaN,2/13/2025,2/16/2025,2000.0,A,51.0,45.0,50.765,45.235,6.0,https://www.surveyusa.com/client/PollReport.as...,-0.732433,316


In [303]:
def avgs_pipeline(data: pd.DataFrame, date: datetime.date):
    state_pivot = pipeline(data)
    # state_pivot['end_date'] = pd.to_datetime(state_pivot['end_date'])
    # state_pivot['pred_plus_minus'] = state_pivot['pred_plus_minus'].map(lambda x: -x)
    state_pivot = state_pivot[state_pivot['end_date'] <= pd.to_datetime(date)]
    state_pivot['population'] = state_pivot['population'].astype(str).map(lambda x: x.strip())
    
    total_num_polls = state_pivot.shape[0]

    # Sample size weights
    total_sample_size = np.sum(state_pivot['sample_size'])
    state_pivot['sample_size_weights'] = (state_pivot['sample_size'].map(lambda x: np.sqrt(min(x, 3000))) / np.sqrt(np.median(state_pivot['sample_size'].map(lambda x: min(x, 2000)))))
    # print(np.median(state_pivot['sample_size'].map(lambda x: min(x, 5000))))
    state_pivot['sample_size_weights'] /= np.sum(state_pivot['sample_size_weights'])
    
    # Time weights
    # Variation of the equation used here: https://polls.votehub.us/
    latest_date = pd.Timestamp(date)
    delta = state_pivot['end_date'].apply(lambda x: (latest_date - x).days)# (latest_date - state_pivot['end_date']).apply(lambda x: x.days)
    linear_weights = (1 - delta/((latest_date - state_pivot['end_date'].min()).days + 1))
    exp_weights = 0.1**(delta/(30))
    state_pivot['time_weights'] =  0 * linear_weights + 1 * exp_weights
    # state_pivot['time_weights'] /= np.sum(state_pivot['time_weights'])
    
    # Quality weights
    rel_quality = state_pivot['pred_plus_minus']
    def quality_weight(rel_qual):
        if rel_qual > 0.5: # Utilizing predictive plus-minus from Silver Bulletin's ratings
            return 0.2
        # return np.sqrt(-rel_qual+1)+0.05
        return np.exp(-rel_qual/1.3)
    state_pivot['quality_weights'] = rel_quality.map(quality_weight)
    # state_pivot['quality_weights'] /= np.sum(state_pivot['quality_weights'])
    
    # Population weights
    def population_weight(population):
        if population == 'RV':
            return 0.8
        elif population == 'LV':
            return 0.6
        return 1
    state_pivot['population_weights'] = state_pivot['population'].map(population_weight)
    # state_pivot['population_weights'] /= np.sum(state_pivot['population_weights'])

    # Gather the weights together
    state_pivot['total_weights'] = state_pivot['sample_size_weights'] * state_pivot['time_weights'] * state_pivot['quality_weights'] * state_pivot['population_weights']
    state_pivot['total_weights'] /= np.sum(state_pivot['total_weights']) # Normalization step
    
    return state_pivot

In [304]:
approval_polls['sample_size'].isna().any()

False

In [305]:
avgs_pipeline(approval_polls, datetime.datetime.today())

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
3,YouGov,Economist,6/13/2025,2025-06-16,1512.0,A,41.0,54.0,41.250,53.750,-13.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,102,0.004985,0.926119,0.964021,1.0,9.524961e-02
13,YouGov,Economist,6/6/2025,2025-06-09,1533.0,A,43.0,52.0,43.250,51.750,-9.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,103,0.005019,0.541170,0.964021,1.0,5.604347e-02
16,YouGov,CBS News,6/4/2025,2025-06-06,2428.0,A,45.0,55.0,45.250,54.750,-10.0,https://www.cbsnews.com/news/deportation-immig...,0.047635,104,0.006317,0.429866,0.964021,1.0,5.602454e-02
22,YouGov,Economist,5/30/2025,2025-06-02,1610.0,A,45.0,49.0,45.250,48.750,-4.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,105,0.005144,0.316228,0.964021,1.0,3.356090e-02
32,YouGov,Economist,5/23/2025,2025-05-26,1660.0,A,44.0,52.0,44.250,51.750,-8.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,106,0.005223,0.184785,0.964021,1.0,1.991321e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,Public Religion Research Institute,NaN,2/28/2025,2025-03-20,5025.0,A,43.0,54.0,43.000,54.000,-11.0,https://www.prri.org/wp-content/uploads/2025/0...,0.103864,311,0.007022,0.001080,0.923213,1.0,1.498062e-04
204,Public Opinion Strategies,Paragon Health Institute,2/25/2025,2025-03-02,1000.0,RV,51.0,49.0,48.930,51.070,2.0,https://paragoninstitute.org/medicaid/paragon-...,-0.246974,312,0.004054,0.000271,1.209225,0.8,2.276484e-05
210,Remington Research Group,Humanity for Freedom Foundation,3/15/2025,2025-03-18,800.0,LV,50.0,48.0,49.295,48.705,2.0,https://www.humanityforfreedomfoundation.org/f...,0.322523,313,0.003626,0.000926,0.780286,0.6,3.364736e-05
287,SurveyUSA,NaN,2/13/2025,2025-02-16,2000.0,A,51.0,45.0,50.765,45.235,6.0,https://www.surveyusa.com/client/PollReport.as...,-0.732433,316,0.005733,0.000093,1.756652,1.0,1.996189e-05


In [306]:
def approval_over_time(data: pd.DataFrame):
    date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
    yes_averages = []
    no_averages = []
    yes_stdevs = []
    no_stdevs = []
    for date in date_range:
        pipelined_df = avgs_pipeline(data, date)
        yes_avg = np.sum(pipelined_df['approve_adj'] * pipelined_df['total_weights'])
        no_avg = np.sum(pipelined_df['disapprove_adj'] * pipelined_df['total_weights'])
        yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve_adj'] - yes_avg)**2))
        no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove_adj'] - no_avg)**2))
        yes_averages.append(yes_avg)
        no_averages.append(no_avg)
        yes_stdevs.append(yes_std)
        no_stdevs.append(no_std)
    
    return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())

In [307]:
approval_avg, approval_std, polls_df = approval_over_time(approval_polls)
approval_avg

,Date,Approve,Disapprove
0,2025-01-23,51.302546,45.716945
1,2025-01-24,51.229190,45.375031
2,2025-01-25,51.229190,45.375031
3,2025-01-26,50.557686,45.329878
4,2025-01-27,49.349467,45.485371
...,...,...,...
141,2025-06-13,45.179099,50.754799
142,2025-06-14,45.179099,50.754799
143,2025-06-15,45.206560,50.772695
144,2025-06-16,44.550386,51.337649


In [308]:
polls_df.shape

(192, 19)

In [309]:
polls_df.head(3)

,pollster,sponsors,startdate,end_date,sample_size,population,approve,disapprove,approve_adj,disapprove_adj,net,url,pred_plus_minus,poll_id,sample_size_weights,time_weights,quality_weights,population_weights,total_weights
3,YouGov,Economist,6/13/2025,2025-06-16,1512.0,A,41.0,54.0,41.25,53.75,-13.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,102,0.004985,0.926119,0.964021,1.0,0.095250
13,YouGov,Economist,6/6/2025,2025-06-09,1533.0,A,43.0,52.0,43.25,51.75,-9.0,https://d3nkl3psvxxpe9.cloudfront.net/document...,0.047635,103,0.005019,0.541170,0.964021,1.0,0.056043
16,YouGov,CBS News,6/4/2025,2025-06-06,2428.0,A,45.0,55.0,45.25,54.75,-10.0,https://www.cbsnews.com/news/deportation-immig...,0.047635,104,0.006317,0.429866,0.964021,1.0,0.056025


In [310]:
polls_df['net_adj'] = polls_df['approve_adj'] - polls_df['disapprove_adj']

In [311]:
display_df = polls_df[['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size', 'population', 'approve', 'disapprove', 'net',
                      'total_weights', 'url']]
display_df['sponsors'] = '^Sponsor: ' + display_df['sponsors'] + '^'
display_df['sponsors'] = display_df['sponsors'].fillna('')
display_df = display_df.sort_values(['end_date'], ascending=False)
display_df = display_df.rename({'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 'total_weights':'Weight', 'startdate':'Start Date', 'end_date':'End Date', 'url':'URL'}, axis='columns')
display_df['Sample'] = display_df['sample_size'].astype(int).astype(str) + ' ' + display_df['population']
# display_df['end_date'] = display_df['end_date'].map(lambda x: datetime.datetime.strptime(datetime.datetime.fromtimestamp(x), '%m/%d/%Y').strftime('%Y-%m-%d'))
# display_df['Date'] = pd.to_datetime(display_df['startdate']).astype(str) + ' - ' + display_df['end_date'].astype(str)
display_df['Start Date'] = pd.to_datetime(display_df['Start Date']).astype(str)
display_df['End Date'] = display_df['End Date'].astype(str)
display_df = display_df.reset_index()
# display_df = display_df.drop(['sample_size', 'population'], axis='columns')
display_df['Pollster'] = '<a href="' + display_df['URL'] + '" style="color:black; text-decoration:underline;"><u>' + display_df['Pollster'] + '</u></a>' + display_df['sponsors']
# display_df['Pollster'] = '[' + display_df['Pollster'] + '](' + display_df['URL'] + ')'
display_df = display_df[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net']]
# display_df = display_df.set_index(['Pollster'])
display_df

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net
0,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-06-13,2025-06-16,1512 A,9.524961e-02,41.0,54.0,-13.000000
1,"<a href=""https://www.reuters.com/world/us/trum...",2025-06-11,2025-06-16,4258 A,9.899232e-02,42.0,54.0,-12.000000
2,"<a href=""https://pro.morningconsult.com/tracke...",2025-06-13,2025-06-15,2207 RV,1.768840e-02,46.0,52.0,-6.000000
3,"<a href=""https://harvardharrispoll.com/wp-cont...",2025-06-11,2025-06-12,2097 RV,6.043487e-02,46.0,50.0,-4.000000
4,"<a href=""https://napolitannews.org/jobapproval...",2025-06-02,2025-06-12,3000 RV,6.613606e-02,52.0,46.0,6.000000
...,...,...,...,...,...,...,...,...
187,"<a href=""https://researchco.ca/2025/01/27/immi...",2025-01-22,2025-01-24,1001 A,2.751574e-07,50.0,46.0,4.000000
188,"<a href=""https://x.com/ScottWRasmussen/status/...",2025-01-20,2025-01-23,3000 RV,1.424858e-06,57.0,39.0,18.000000
189,"<a href=""https://cdn.atlasintel.org/ed3f5238-e...",2025-01-21,2025-01-23,1882 A,3.649770e-06,50.1,49.7,0.399998
190,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,8.062729e-07,47.0,41.0,6.000000


In [312]:
display_df.to_csv('polls_display_table.csv')

In [313]:
approval_std

,Date,Approve,Disapprove
0,2025-01-23,3.697758,5.575750
1,2025-01-24,3.393680,5.186733
2,2025-01-25,3.393680,5.186733
3,2025-01-26,3.666409,4.754769
4,2025-01-27,3.674859,4.356649
...,...,...,...
141,2025-06-13,3.672679,3.259397
142,2025-06-14,3.672679,3.259397
143,2025-06-15,3.636765,3.225635
144,2025-06-16,3.536327,3.115858


In [314]:
approval_avg['Net'] = approval_avg['Approve'] - approval_avg['Disapprove']
approval_avg

,Date,Approve,Disapprove,Net
0,2025-01-23,51.302546,45.716945,5.585601
1,2025-01-24,51.229190,45.375031,5.854159
2,2025-01-25,51.229190,45.375031,5.854159
3,2025-01-26,50.557686,45.329878,5.227807
4,2025-01-27,49.349467,45.485371,3.864096
...,...,...,...,...
141,2025-06-13,45.179099,50.754799,-5.575700
142,2025-06-14,45.179099,50.754799,-5.575700
143,2025-06-15,45.206560,50.772695,-5.566135
144,2025-06-16,44.550386,51.337649,-6.787262


In [315]:
approval_trendline = pd.merge(left=approval_avg, right=approval_std, left_on='Date', right_on='Date', how='inner')
approval_trendline['approve_CI_upper'] = approval_trendline['Approve_x'] + 1.96*approval_trendline['Approve_y']
approval_trendline['approve_CI_lower'] = approval_trendline['Approve_x'] - 1.96*approval_trendline['Approve_y']
approval_trendline['disapprove_CI_upper'] = approval_trendline['Disapprove_x'] + 1.96*approval_trendline['Disapprove_y']
approval_trendline['disapprove_CI_lower'] = approval_trendline['Disapprove_x'] - 1.96*approval_trendline['Disapprove_y']
approval_trendline = approval_trendline.rename({
    'Approve_x':'approve_mean',
    'Disapprove_x':'disapprove_mean',
}, axis='columns').drop(['Approve_y', 'Disapprove_y', 'Net'], axis='columns')
approval_trendline

,Date,approve_mean,disapprove_mean,approve_CI_upper,approve_CI_lower,disapprove_CI_upper,disapprove_CI_lower
0,2025-01-23,51.302546,45.716945,58.550152,44.054940,56.645416,34.788474
1,2025-01-24,51.229190,45.375031,57.880803,44.577578,55.541028,35.209034
2,2025-01-25,51.229190,45.375031,57.880803,44.577578,55.541028,35.209034
3,2025-01-26,50.557686,45.329878,57.743848,43.371524,54.649225,36.010531
4,2025-01-27,49.349467,45.485371,56.552191,42.146743,54.024404,36.946338
...,...,...,...,...,...,...,...
141,2025-06-13,45.179099,50.754799,52.377549,37.980649,57.143218,44.366381
142,2025-06-14,45.179099,50.754799,52.377549,37.980649,57.143218,44.366381
143,2025-06-15,45.206560,50.772695,52.334618,38.078501,57.094940,44.450450
144,2025-06-16,44.550386,51.337649,51.481587,37.619185,57.444730,45.230567


In [316]:
# Export approval ratings
approval_trendline.to_csv('approval_stats.csv')

In [317]:
polls_df.shape

(192, 20)

In [318]:
line = px.line(approval_avg, x='Date', y=['Approve', 'Disapprove'])
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['approve', 'disapprove'])
line.update_traces(hovertemplate=None)
line.update_layout(hovermode='x unified')
# scatter.update_traces(hovertemplate=None, hoverinfo='skip')
fig_approve_CI = go.Figure([
    go.Scatter(
        name='Approve CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] + 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Approve CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Approve'] - 1.96*approval_std['Approve'],
        mode='lines',
        marker=dict(color='#8be8f2'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])
fig_disapprove_CI = go.Figure([
    go.Scatter(
        name='Disapprove CI Upper Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] + 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    go.Scatter(
        name='Disapprove CI Lower Bound',
        x = approval_avg['Date'],
        y = approval_avg['Disapprove'] - 1.96*approval_std['Disapprove'],
        mode='lines',
        marker=dict(color='#a3cbec'),
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),  
])

fig = go.Figure(data=line.data + fig_approve_CI.data + fig_disapprove_CI.data)
fig.update_layout(
    title='President Trump Approval Rating',
    xaxis_title='Date',
    yaxis_title='%',
    legend_title='Legend',
    hovermode='x unified'
)
fig.show()

In [319]:
line = px.line(approval_avg, x='Date', y=['Net'])
scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net'])
                     #trendline='lowess', trendline_options=dict(frac=0.2))
fig = go.Figure(data=line.data + scatter.data)
fig.add_hline(y=0, line_dash='dash')
fig.show()

In [320]:
# scatter = px.scatter(polls_df[polls_df['end_date'] >= pd.to_datetime('2025-01-24')], x='end_date', y=['net', 'net_adj'])
# fig = go.Figure(data=scatter.data)
# fig.add_hline(y=0, line_dash='dash')
# fig.show()

In [321]:
def net_col(df: pd.DataFrame, min_date: datetime.date):
#     df_copy = df.set_index(['Date'])
#     return df_copy['approve_mean'] - df_copy['disapprove_mean']
    date_range = pd.date_range(start=min_date, end=datetime.datetime.today(), freq='d', inclusive='both')
    data = df.copy()
    data['net_adj'] = data['approve_adj'] - data['disapprove_adj']
    net_averages = []
    net_stdevs = []
    issue_cols = {'Date': date_range, 'net':[], 'net_std':[]}
    for date in date_range:
    #         try:
            pipelined_df = avgs_pipeline(data, date)
    #         except TypeError:
    #             print('TypeError')
    #             print(date)
            net_avg = np.sum(pipelined_df['net_adj'] * pipelined_df['total_weights'])
            net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net_adj'] - net_avg)**2))

            net_averages.append(net_avg)
            net_stdevs.append(net_std)

    issue_cols['net'].extend(net_averages)
    issue_cols[f'net_std'].extend(net_stdevs)
        
    net_averages, net_stdevs = [], []
        
    return pd.DataFrame(issue_cols)

In [322]:
net_approval = net_col(approval, datetime.date(2025, 1, 23))
net_approval

,Date,net,net_std
0,2025-01-23,5.585601,8.638245
1,2025-01-24,5.854159,7.957334
2,2025-01-25,5.854159,7.957334
3,2025-01-26,5.227807,7.578410
4,2025-01-27,3.864096,6.999819
...,...,...,...
141,2025-06-13,-5.575700,6.128189
142,2025-06-14,-5.575700,6.128189
143,2025-06-15,-5.566135,6.060887
144,2025-06-16,-6.787262,5.988566


In [323]:
net_approval['net_upper_CI'] = net_approval['net'] + 1.96*net_approval['net_std']
net_approval['net_lower_CI'] = net_approval['net'] - 1.96*net_approval['net_std']
net_approval

,Date,net,net_std,net_upper_CI,net_lower_CI
0,2025-01-23,5.585601,8.638245,22.516561,-11.345359
1,2025-01-24,5.854159,7.957334,21.450534,-9.742216
2,2025-01-25,5.854159,7.957334,21.450534,-9.742216
3,2025-01-26,5.227807,7.578410,20.081490,-9.625875
4,2025-01-27,3.864096,6.999819,17.583741,-9.855549
...,...,...,...,...,...
141,2025-06-13,-5.575700,6.128189,6.435550,-17.586951
142,2025-06-14,-5.575700,6.128189,6.435550,-17.586951
143,2025-06-15,-5.566135,6.060887,6.313204,-17.445475
144,2025-06-16,-6.787262,5.988566,4.950326,-18.524851


In [324]:
net_approval.to_csv('net_approval.csv')

### Auxiliary Graph: Individual Pollsters

In [325]:
approval_polls.columns.values

array(['pollster', 'sponsors', 'startdate', 'end_date', 'sample_size',
       'population', 'approve', 'disapprove', 'approve_adj',
       'disapprove_adj', 'net', 'url', 'pred_plus_minus', 'poll_id'],
      dtype=object)

In [326]:
appr = full_approval.copy()
appr['net_adj'] = appr['approve'] - appr['disapprove']
pollsters = appr['pollster'].value_counts().index.values
pollsters

array(['Rasmussen Reports', 'YouGov', 'Morning Consult', 'RMG Research',
       'Ipsos', 'J.L. Partners', 'TIPP Insights',
       'Global Strategy Group/GBAO (Navigator Research)',
       'Harris Insights & Analytics', 'Echelon Insights',
       'Quinnipiac University', 'AtlasIntel', 'Emerson College', 'Gallup',
       'McLaughlin & Associates', 'SSRS', 'American Research Group',
       'co/efficient', 'Civiqs', 'John Zogby Strategies', 'Cygnal',
       'Clarity Campaign Labs', 'Marquette University Law School',
       'SurveyMonkey', 'InsiderAdvantage', 'Marist College',
       'Hart Research Associates/Public Opinion Strategies',
       'Pew Research Center', 'Fabrizio Lee & Associates/Impact Research',
       'Blueprint Polling', 'Beacon Research/Shaw & Co. Research',
       'Public Religion Research Institute', 'SurveyUSA',
       'OnMessage Inc.', 'North Star Opinion Research',
       'Remington Research Group', 'Public Opinion Strategies',
       'OH Predictive Insights', 'GBAO',

In [327]:
def approval_today(data: pd.DataFrame):
    # date_range = pd.date_range(start=datetime.date(2025, 1, 23), end=datetime.datetime.today(), freq='d', inclusive='both')
#     yes_averages = []
#     no_averages = []
#     yes_stdevs = []
#     no_stdevs = []
    pipelined_df = avgs_pipeline(data, datetime.datetime.today())
    yes_avg = np.sum(pipelined_df['approve'] * pipelined_df['total_weights'])
    no_avg = np.sum(pipelined_df['disapprove'] * pipelined_df['total_weights'])
    yes_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['approve'] - yes_avg)**2))
    no_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['disapprove'] - no_avg)**2))
    net_avg = np.sum(pipelined_df['net'] * pipelined_df['total_weights'])
    net_std = np.sqrt(np.sum(pipelined_df['total_weights'] * (pipelined_df['net'] - net_avg)**2))
    
    #return pd.DataFrame({'Date':date_range, 'Approve':yes_averages, 'Disapprove':no_averages}), pd.DataFrame({'Date':date_range, 'Approve':yes_stdevs, 'Disapprove':no_stdevs}), avgs_pipeline(data, datetime.datetime.today())
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [328]:
def indiv_pollster_avg(pollster: str):
    df = appr[appr['pollster'] == pollster]
    yes_avg, no_avg, yes_std, no_std, net_avg, net_std = approval_today(df)
    return yes_avg, no_avg, yes_std, no_std, net_avg, net_std

In [329]:
def get_ppm(pollster: str):
    df = ratings[ratings['Pollster'] == pollster]
    ppm = df['Predictive Plus-Minus'].values[0]
    
    return ppm

In [330]:
appr.shape

(323, 26)

In [331]:
def get_num_polls(pollster: str):
    df = appr[appr['pollster'] == pollster]
    num_polls = df.shape[0]
    return num_polls

In [332]:
get_ppm('YouGov')

0.047635015

In [333]:
indiv_pollster_avg('YouGov')

(42.900474191057825,
 52.76860451019764,
 1.6696324139560228,
 1.9805638285980822,
 -9.868130319139828,
 2.9506225667562145)

In [334]:
indiv_pollster_avg('RMG Research')

(50.8148284482443,
 47.19053084358856,
 1.436303210310549,
 1.7228707035020834,
 3.6242976046557227,
 3.075291456018617)

In [335]:
nets = []
net_stdevs = []
ppms = []
nums = []

for p in pollsters:
    _, _, _, _, net, net_std = indiv_pollster_avg(p)
    ppm = get_ppm(p)
    num_polls = get_num_polls(p)
    
    nets.append(net)
    net_stdevs.append(net_std)
    ppms.append(ppm)
    nums.append(num_polls)

In [336]:
# nets

In [337]:
indiv_avgs = pd.DataFrame({'Pollster': pollsters, 'Net Approval': nets, 'Net Stdev': net_stdevs, 'Predictive Plus-Minus': ppms, 'Number of Approval Polls Conducted': nums})
indiv_avgs = indiv_avgs.sort_values(['Net Approval'], ascending=True)
indiv_avgs

,Pollster,Net Approval,Net Stdev,Predictive Plus-Minus,Number of Approval Polls Conducted
27,Pew Research Center,-18.930698,1.017217,0.132230,2
15,SSRS,-17.494832,2.384042,0.966835,4
10,Quinnipiac University,-15.959728,0.434969,-0.290484,5
16,American Research Group,-13.567189,1.349999,0.818141,4
7,Global Strategy Group/GBAO (Navigator Research),-12.410504,1.263552,0.804988,7
41,The New York Times/Siena College,-12.000000,0.000000,-0.763947,1
40,NewsNation,-12.000000,0.000000,0.614459,1
4,Ipsos,-11.743412,1.023970,0.442898,15
31,Public Religion Research Institute,-11.000000,0.000000,0.103864,1
25,Marist College,-10.898258,0.837759,-0.436610,2


In [338]:
indiv_avgs.to_csv('indiv_pollster_avgs.csv')

In [339]:
reg = lm.LinearRegression()
reg.fit(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
reg.coef_, reg.intercept_

(array([-1.29698781]), -5.190963907810294)

In [340]:
r = r_regression(indiv_avgs[['Predictive Plus-Minus']], indiv_avgs['Net Approval'])
r

array([-0.109654])